In [ ]:
!pip install sympy numpy pandas matplotlib scipy ipython
# this should take up to 7 minutes

In [1]:
from sympy import *
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import fsolve
from IPython.display import display, Math, Latex

In [ ]:
def crate_symbol(name):
    return symbols(name, real=True, positive=True)

        
u = crate_symbol("u")

x1 = crate_symbol("x1")
x2 = crate_symbol("x2")
x3 = crate_symbol("x3")
x4 = crate_symbol("x4")
x5 = crate_symbol("x5")

a1 = crate_symbol("a1")
a2 = crate_symbol("a2")
a3 = crate_symbol("a3")
a4 = crate_symbol("a4")
a5 = crate_symbol("a5")

b1 = crate_symbol("b1")
b2 = crate_symbol("b2")
b3 = crate_symbol("b3")
b4 = crate_symbol("b4")
b5 = crate_symbol("b5")

K_GR = crate_symbol("K_GR")
H1 = crate_symbol("H1")
H2 = crate_symbol("H2")
P = crate_symbol("P")
A = crate_symbol("A")
t = crate_symbol("t")

aP = crate_symbol("aP")
aA = crate_symbol("aA")

bP = crate_symbol("bP")
bA = crate_symbol("bA")


mr = 1 / x3
gr = K_GR ** 3 / (x3 ** 3 + K_GR ** 3)
# # gr = 1 / (1 + (x_3 / K_GR) ** 3)
# gr = 1
f = gr * mr
dx1_dt = b1 * u * gr * mr * x5 - a1 * x1
dx2_dt = b2 * x1 * P * gr * x5- a2 * x2
dx3_dt = b3 * x2 * A *x4 * x5 - a3 * x3
dx4_dt = b4 * u -  a4 * x4
dx5_dt = b5 * u * gr - a5 * x5
dP_dt = P * (bP * x1 * x4 - aP)
dA_dt = A * (bA * x2 - aA)

display(
    Matrix(
        [
            Eq(Derivative(x1, t), dx1_dt),
            Eq(Derivative(x2, t), dx2_dt),
            Eq(Derivative(x3, t), dx3_dt),
            Eq(Derivative(x4, t), dx4_dt),
            Eq(Derivative(x5, t), dx5_dt),
        ]
    )
)
model = Matrix([dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt, dP_dt, dA_dt])
variables = (x1, x2, x3, x4, x5, P, A)

In [3]:
parameters = {
    u: 1,
    a1: 1,
    a2: 1,
    a3: 1,
    a4: 1,
    a5: 1,
    b1: 1,
    b2: 1,
    b3: 1,
    b4: 1,
    b5: 1,
    K_GR: 4,
    H1: 1,
    H2: 1,
    aA: 1,
    aP: 1,
    bA: 1,
    bP: 1,
    # P: 1,
    # A: 1,
    # x1: 1,
    # x2: 1,
    # x3: 1,
    # x4: 1,
    # x5: 1,
}

In [ ]:
def run_sim(model, variables,parameters, time, high_u_times=[]):
    model_lambda = lambdify(variables, model.subs(parameters))
    if u in parameters:
        parameters.pop(u)
    def model_ode(y, t):
        for high_u_time in high_u_times:
            if t > high_u_time[0] and t < high_u_time[1]:
                parameters[u] = 2
            else:
                parameters[u] = 1
        return model_lambda(*y).flatten()
    y0 = [1 for _ in variables]
    y = odeint(model_ode, y0, time)
    y_df = pd.DataFrame(y, columns=[str(v) for v in variables])
    return y_df
t = np.linspace(0, 100, 1000)
variables = (u,x1, x2, x3, x4, x5, P, A)
solution  = run_sim(model, variables, parameters, t)
plt.figure(figsize=(10, 8))
plt.plot(t, solution[:, 0], label='x1')
plt.plot(t, solution[:, 1], label='x2')
# plt.plot(t, solution[:, 2], label='x3')
plt.plot(t, solution[:, 3], label='x4')
plt.plot(t, solution[:, 4], label='x5')
plt.plot(t, solution[:, 5], label='P')
plt.plot(t, solution[:, 6], label='A')
plt.xlabel('Time')
plt.ylabel('Variables')
plt.legend()

    

In [ ]:
solution.plot(backend="plotly")

In [ ]:
lambdify(variables, model.subs(parameters)) (*([1] * 7 ))

In [ ]:
model

In [ ]:
# Define the system of differential equations with piecewise function for u
def model_system(y, t, params):
    x1, x2, x3, x4, x5, P,A = y
    a1, a2, a3, a4, a5, b1, b2, b3, b4, b5, K_GR = params
    # print(params)
    # Define u as a piecewise function
    u = 1 if t < 15 or (t > 20 and t < 35) or t > 40 else 2
    K_GR = 4 if t < 30 else 2
    # gr.subs({K_GR:4})
    
    # u = 1 if t < 15 and t <20 else 2
    
    mr = 1 / x3
    gr = 1 / (1 + (x3 / K_GR) ** 3)
    f = gr * mr
    
    dx1_dt = b1 * u * gr * mr - a1 * x1
    dx2_dt = b2 * x1 * P * gr - a2 * x2
    dx3_dt = b3 * x2 * A * x4  - a3 * x3
    dx4_dt = b4 * u * gr - a4 * x4
    dx5_dt = b5 * u * gr - a5 * x5
    dP_dt = P*(x1 * x4 - 1)
    dA_dt = A* (x2  - 1)
    

    # dx1_dt = b1 * u * gr * mr * x5 - a1 * x1
    # dx2_dt = b2 * x1 * P * gr * x5 - a2 * x2
    # dx3_dt = b3 * x2 * A * x4 * x5 * gr - a3 * x3
    # dx4_dt = b4 * u * gr - a4 * x4
    # dx5_dt = b5 * u * gr - a5 * x5
    # print(dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt, dP_dt, dA_dt)
    return [dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt, dP_dt, dA_dt]

# Initial conditions
y0 = [1, 1, 1, 1, 1,1,1]

# Time points where solution is computed
t = np.linspace(0, 60, 400)

# Parameters
params = [parameters[a1], parameters[a2], parameters[a3], parameters[a4], parameters[a5],
          parameters[b1], parameters[b2], parameters[b3], parameters[b4], parameters[b5], parameters[K_GR]]

# Solve the system of differential equations
solution = odeint(model_system, y0, t, args=(params,))

# Plot the results
plt.figure(figsize=(10, 8))
plt.plot(t, solution[:, 0], label='x1')
plt.plot(t, solution[:, 1], label='x2')
# plt.plot(t, solution[:, 2], label='x3')
plt.plot(t, solution[:, 3], label='x4')
plt.plot(t, solution[:, 4], label='x5')
plt.plot(t, solution[:, 5], label='P')
plt.plot(t, solution[:, 6], label='A')
plt.xlabel('Time')
plt.ylabel('Variables')
plt.legend()
plt.title('Simulation of Stress with u changing at t=5')
plt.show()

In [ ]:
# Define the system of differential equations with piecewise function for u
def model_system(y, t, params):
    x1, x2, x3, x4, x5, P,A = y
    a1, a2, a3, a4, a5, b1, b2, b3, b4, b5, K_GR = params
    
    # Define u as a piecewise function
    u = 1 if t < 15 or (t > 20 and t < 35) or t > 40 else 2
    K_GR = 4 if t < 30 else 2
    
    # u = 1 if t < 15 and t <20 else 2
    
    mr = 1 / x3
    # gr = 1 / (1 + (x3 / K_GR) ** 3)
    f = gr * mr
    
    dx1_dt = b1 * u * gr * mr - a1 * x1
    dx2_dt = b2 * x1 * P * gr - a2 * x2
    dx3_dt = b3 * x2 * A * x4  - a3 * x3
    dx4_dt = b4 * u * gr - a4 * x4
    dx5_dt = b5 * u * gr - a5 * x5
    dP_dt = P(x1 * x_4 - 1)
    dA_dt = A(x2  - 1)
    

    # dx1_dt = b1 * u * gr * mr * x5 - a1 * x1
    # dx2_dt = b2 * x1 * P * gr * x5 - a2 * x2
    # dx3_dt = b3 * x2 * A * x4 * x5 * gr - a3 * x3
    # dx4_dt = b4 * u * gr - a4 * x4
    # dx5_dt = b5 * u * gr - a5 * x5
    
    return [dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt, dP_dt, dA_dt]

# Initial conditions
y0 = [1, 1, 1, 1, 1,1,1]

# Time points where solution is computed
t = np.linspace(0, 60, 400)

# Parameters
params = [parameters[a1], parameters[a2], parameters[a3], parameters[a4], parameters[a5],
          parameters[b1], parameters[b2], parameters[b3], parameters[b4], parameters[b5],
          parameters[K_GR]]

# Solve the system of differential equations
solution = odeint(model_system, y0, t, args=(params,))

# Plot the results
plt.figure(figsize=(10, 8))
plt.plot(t, solution[:, 0], label='x1')
plt.plot(t, solution[:, 1], label='x2')
plt.plot(t, solution[:, 2], label='x3')
plt.plot(t, solution[:, 3], label='x4')
plt.plot(t, solution[:, 4], label='x5')
plt.plot(t, solution[:, 5], label='P')
plt.plot(t, solution[:, 6], label='A')
plt.xlabel('Time')
plt.ylabel('Variables')
plt.legend()
plt.title('Simulation of Stress with u changing at t=5')
plt.show()

In [ ]:
model.subs(parameters)

In [ ]:
model

In [ ]:
def model_fixed_point_function(ic, model_function):
    return model_function(*ic).flatten()

guess = [
    1,  # u
    1,  # x_1
    1,  # x_2
    1,  # x_3
    1,  # P
]
# use lsoda to solve the system
parameters[u] = 1
parameters[x5] = 1
fix_point = solve(model.subs(parameters)[:2], variables)
for point in fix_point:
    for key in point:
        display(key)
# model_f = lambdify(variables, model.subs(parameters))
# ic = fsolve(model_fixed_point_function, guess, args=model_f)
# ic